In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
from tensorflow import keras
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


from input_generator import load_dataset, load_dataset_with_lables
from data_parser import get_tags_and_labels, get_vocab, get_tags
from df import create_seperate_columns, create_listed_columns

d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [2]:
# Get dataframe with tags and category_id columns
csvpath = 'data/CAvideos.cvs'
tab_dataframe = get_tags_and_labels(csvpath)
max_of_tags = 124
tab_dataframe = create_seperate_columns(tab_dataframe, max_of_tags, 'tags')

tab_dataframe.head()

,category_id,tags0,tags1,tags2,tags3,tags4,tags5,tags6,tags7,tags8,...,tags114,tags115,tags116,tags117,tags118,tags119,tags120,tags121,tags122,tags123
0,10,Eminem,Walk,On,Water,Aftermath/Shady/Interscope,Rap,notags,notags,notags,...,notags,notags,notags,notags,notags,notags,notags,notags,notags,notags
1,23,plush,bad unboxing,unboxing,fan mail,idubbbztv,idubbbztv2,things,best,packages,...,notags,notags,notags,notags,notags,notags,notags,notags,notags,notags
2,23,racist superman,rudy,mancuso,king,bach,racist,superman,love,rudy mancuso poo bear black white official mus...,...,notags,notags,notags,notags,notags,notags,notags,notags,notags,notags
3,24,ryan,higa,higatv,nigahiga,i dare you,idy,rhpc,dares,no truth,...,notags,notags,notags,notags,notags,notags,notags,notags,notags,notags
4,10,edsheeran,ed sheeran,acoustic,live,cover,official,remix,official video,lyrics,...,notags,notags,notags,notags,notags,notags,notags,notags,notags,notags


In [3]:
# Getting leaning vocab
vocab = get_vocab(get_tags(csvpath))

# Turn vocab to dictionary
voc_di = {i: vo for i, vo in enumerate(vocab)}
voc_di[0] = 'notags'
voc_di = {vo: i for i, vo in voc_di.items()}

# Mapping tags to indexes in vocab
for index in range(0, max_of_tags):
    tab_dataframe['tags{}'.format(index)] = tab_dataframe['tags{}'.format(index)].map(voc_di)

tab_dataframe.fillna(0.0, inplace = True) 
tab_dataframe.head()

,category_id,tags0,tags1,tags2,tags3,tags4,tags5,tags6,tags7,tags8,...,tags114,tags115,tags116,tags117,tags118,tags119,tags120,tags121,tags122,tags123
0,10,52579.0,148216.0,22297.0,32998.0,128430.0,41098.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,23,50620.0,51100.0,127040.0,110579.0,115493.0,57712.0,120864.0,10309.0,108079.0,...,0,0,0,0,0,0,0,0,0,0
2,23,35336.0,46149.0,133327.0,50019.0,55494.0,141360.0,102197.0,124593.0,40953.0,...,0,0,0,0,0,0,0,0,0,0
3,24,101711.0,48661.0,107051.0,26719.0,113818.0,48115.0,113544.0,111307.0,15029.0,...,0,0,0,0,0,0,0,0,0,0
4,10,144975.0,54276.0,38220.0,120137.0,105716.0,93177.0,1115.0,134050.0,42465.0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
future_df = []
col_row_dict = {}

for i in range(0, 40881):
    for col in tab_dataframe.columns:
        if col == 'category_id':
            col_row_dict.update({'category_id':tab_dataframe[col][i]})
        else:
            col_row_dict.update({'tag{}'.format(tab_dataframe[col][i]):1.0})
    future_df.append(col_row_dict)
    col_row_dict = {}

In [17]:
voc_fd = pd.DataFrame(future_df, dtype=float)
voc_fd.fillna(0.0, inplace = True)
voc_fd.head()

TypeError: __init__() got an unexpected keyword argument 'low_memory'

In [ ]:
# Create train,validation and test examples from dataframe
# train_ds, val_ds, test_ds = load_dataset(tab_dataframe)
(train_ds, train_lb), (val_ds, val_lb), (test_ds, test_lb) = load_dataset_with_lables(voc_fd, 'category_id')


In [ ]:
# first 5 training examples
print(train_ds.values[5:10])
train_lb.values[0:5]

In [ ]:
hidden = 64
model = tf.keras.Sequential([
    layers.Flatten(),
    layers.Dense(hidden,activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.Dense(2*hidden,activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.Dense(44, activation='softmax')])

model.compile(optimizer='nadam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              )

history = model.fit(train_ds.values, train_lb.values,
                    batch_size=32,
                    epochs=20,
                    validation_data=(val_ds, val_lb))
